In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read the crime data
la_crime = pd.read_csv('Crime_Data_2010_2017.csv')

In [3]:
# List of columns to drop
columns_to_drop = ['DR Number', 'Area ID', 'Reporting District', 'MO Codes', 'Premise Code', 'Status Code', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Cross Street']

# Drop the columns
la_crime = la_crime.drop(columns=columns_to_drop)

In [4]:
# Rename columns
column_rename_map = {
    'Date Reported': 'date_reported',
    'Date Occurred': 'date_occured',
    'Time Occurred': 'time_occurred',
    'Area Name': 'area_name',
    'Crime Code Description': 'crime_code_description',
    'Victim Age': 'victim_age',
    'Victim Sex': 'victim_sex',
    'Victim Descent': 'victim_descent',
    'Premise Description': 'premise_description',
    'Weapon Description': 'weapon_description',
    'Status Description': 'status_description'
}
la_crime = la_crime.rename(columns=column_rename_map)


In [5]:
# Format the Time and Date columns
la_crime['date_reported'] = pd.to_datetime(la_crime['date_reported'], format='%m/%d/%Y')
la_crime['date_occured'] = pd.to_datetime(la_crime['date_occured'], format='%m/%d/%Y')

la_crime['hour'] = (la_crime['time_occurred'] // 100).astype(int)
la_crime['minute'] = (la_crime['time_occurred'] % 100).astype(int)

la_crime = la_crime.drop(columns=['time_occurred'])

la_crime['month'] = la_crime['date_occured'].dt.month
la_crime['weekday'] = la_crime['date_occured'].dt.day_name()

def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

la_crime['season'] = la_crime['month'].apply(get_season)

In [6]:
la_crime.head(5)

,date_reported,date_occured,area_name,Crime Code,crime_code_description,victim_age,victim_sex,victim_descent,premise_description,Weapon Used Code,weapon_description,status_description,Address,Location,hour,minute,month,weekday,season
0,2013-03-14,2013-03-11,77th Street,626,INTIMATE PARTNER - SIMPLE ASSAULT,30.0,F,W,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Adult Other,6300 BRYNHURST AV,"(33.9829, -118.3338)",18,0,3,Monday,Spring
1,2010-01-25,2010-01-22,Olympic,510,VEHICLE - STOLEN,NaN,NaN,NaN,STREET,NaN,NaN,Invest Cont,VAN NESS,"(34.0454, -118.3157)",23,0,1,Friday,Winter
2,2013-03-19,2013-03-18,Southeast,510,VEHICLE - STOLEN,12.0,NaN,NaN,STREET,NaN,NaN,Invest Cont,200 E 104TH ST,"(33.942, -118.2717)",20,30,3,Monday,Spring
3,2010-11-11,2010-11-10,Southeast,510,VEHICLE - STOLEN,NaN,NaN,NaN,STREET,NaN,NaN,Invest Cont,88TH,"(33.9572, -118.2717)",18,0,11,Wednesday,Fall
4,2014-01-11,2014-01-04,Topanga,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),84.0,M,W,SINGLE FAMILY DWELLING,NaN,NaN,Invest Cont,7200 CIRRUS WY,"(34.2009, -118.6369)",23,0,1,Saturday,Winter


In [7]:
# change format of location into longitude and latitude
la_crime[['longitude', 'latitude']] = la_crime['Location '].str.split(',', expand=True)

la_crime['longitude'] = la_crime['longitude'].str[1:]
la_crime['latitude'] = la_crime['latitude'].str[:-1]

la_crime['longitude'] = pd.to_numeric(la_crime['longitude'])
la_crime['latitude'] = pd.to_numeric(la_crime['latitude'])

la_crime = la_crime.drop(columns=['Location '])

In [8]:
# for longitude and latitude, we decided to remove the columns with missing values since there are only 9 missing value out of 1584316 
# for crime_code_description, victim_age, victim_descent, we changed the missing values with 'unknown'
# for victim_age, we changed the missing values with integer 999
la_crime = la_crime.dropna(subset=['longitude', 'latitude'])
la_crime['crime_code_description'].fillna('unknown', inplace=True)
la_crime['victim_age'].fillna(999, inplace=True)
la_crime['victim_sex'].fillna('unknown', inplace=True)
la_crime['victim_descent'].fillna('unknown', inplace=True)
la_crime['premise_description'].fillna('no description', inplace=True)
la_crime['weapon_description'].fillna('no description', inplace=True)

In [9]:
# Format the vicim_sex column
victim_sex_mapping = {
    "F": "Female",
    "M": "Male",
    "H": "Heterosexuality",
    "X": "Indeterminate"
}

# Apply the mapping to the 'victim_sex' column
la_crime['victim_sex'] = la_crime['victim_sex'].replace(victim_sex_mapping)

In [10]:
# Format the victim_descent column
# Mapping of codes to descents
descent_mapping = {
    "W": "White",
    "H": "Hispanic",
    "B": "Black or African American",
    "O": "Other",
    "A": "Asian",
    "X": "Unknown",
    "F": "Filipino",
    "I": "American Indian or Alaska Native",
    "G": "Guamanian",
    "C": "Chinese",
    "J": "Japanese",
    "K": "Korean",
    "P": "Pacific Islander",
    "V": "Vietnamese",
    "U": "Hawaiian",
    "L": "Laotian",
    "Z": "Asian Indian",
    "S": "Samoan",
    "D": "Cambodian",
    "-": "Not Specified"
}

# Replace the codes with the actual descents in the 'victim_descent' column
la_crime['victim_descent'] = la_crime['victim_descent'].replace(descent_mapping)

In [11]:
la_crime.head(5)

,date_reported,date_occured,area_name,Crime Code,crime_code_description,victim_age,victim_sex,victim_descent,premise_description,Weapon Used Code,weapon_description,status_description,Address,hour,minute,month,weekday,season,longitude,latitude
0,2013-03-14,2013-03-11,77th Street,626,INTIMATE PARTNER - SIMPLE ASSAULT,30.0,Female,White,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",Adult Other,6300 BRYNHURST AV,18,0,3,Monday,Spring,33.9829,-118.3338
1,2010-01-25,2010-01-22,Olympic,510,VEHICLE - STOLEN,999.0,unknown,unknown,STREET,NaN,no description,Invest Cont,VAN NESS,23,0,1,Friday,Winter,34.0454,-118.3157
2,2013-03-19,2013-03-18,Southeast,510,VEHICLE - STOLEN,12.0,unknown,unknown,STREET,NaN,no description,Invest Cont,200 E 104TH ST,20,30,3,Monday,Spring,33.9420,-118.2717
3,2010-11-11,2010-11-10,Southeast,510,VEHICLE - STOLEN,999.0,unknown,unknown,STREET,NaN,no description,Invest Cont,88TH,18,0,11,Wednesday,Fall,33.9572,-118.2717
4,2014-01-11,2014-01-04,Topanga,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),84.0,Male,White,SINGLE FAMILY DWELLING,NaN,no description,Invest Cont,7200 CIRRUS WY,23,0,1,Saturday,Winter,34.2009,-118.6369


In [12]:
# Simple format for the crime_code_description column
la_crime['crime_code_description'] = la_crime['crime_code_description'].str.title().str.strip()
la_crime['crime_code_description'] = la_crime['crime_code_description'].replace({'unknown': 'Unknown'})

In [13]:
# If we only need the rows with top 10 most common crime code description, we can use the following code
top_10_crimes = la_crime['crime_code_description'].value_counts().head(10).index
print(top_10_crimes)
la_crime_top_10_crimes = la_crime[la_crime['crime_code_description'].isin(top_10_crimes)]

Index(['Battery - Simple Assault', 'Vehicle - Stolen', 'Burglary From Vehicle',
       'Burglary', 'Theft Plain - Petty ($950 & Under)', 'Theft Of Identity',
       'Intimate Partner - Simple Assault',
       'Vandalism - Felony ($400 & Over, All Church Vandalisms) 0114',
       'Vandalism - Misdeameanor ($399 Or Under)',
       'Assault With Deadly Weapon, Aggravated Assault'],
      dtype='object')


In [14]:
# Add ID column
la_crime['ID'] = range(len(la_crime))
# save to csv
la_crime.to_csv('la_crime_cleaned.csv', index=False)

In [15]:
# Add ID column for la_crime_top_10_crimes
la_crime_top_10_crimes['ID'] = range(len(la_crime_top_10_crimes))
# save to csv
la_crime_top_10_crimes.to_csv('la_crime_top_10_crimes_cleaned.csv', index=False)

<ipython-input-15-68bd9f310ee5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_crime_top_10_crimes['ID'] = range(len(la_crime_top_10_crimes))
